In [17]:
import numpy as np
import pandas as pd
import sys
import getpass
import re
from functions import *
from collections import Counter
np.set_printoptions(precision=2)
pd.set_option('precision', 2)

In [18]:
df = pd.read_csv('C:\Users\\'+getpass.getuser()+'\\Google Drive\Sarah Ido folder\data\CCK\MATCHING_phet_cck_user_actions+sophistication_WITHPAUSE_anonymized.txt')
df_scores = pd.read_csv('C:\Users\\'+getpass.getuser()+'\\Google Drive\Sarah Ido folder\data\CCK\MATCHING_phet_cck_user_data_anonymized.txt')
df["student"] = df["student"].astype('category')
df_scores["student"] = df_scores["student"].astype('category')
df["Family"]=df["Family"].str.capitalize()

Let's get the sequences by students in blocks of actions.

In [40]:
def get_blocks(df, students):
    '''gets blocks of sequences a list of students'''
    blocks = {student:'S' for student in students}
    for student in students:
        sequence =  list(df[df['student']==student]['Family'])
        blocks[student] += re.sub(r'(.)\1+', r'\1',''.join([action[0] if action != 'Reset' else 'X' for action in sequence]))
    return blocks

def get_frequencies(blocks):
    frequencies = {}
    for student,sequence in blocks.iteritems():
        for seq_length in range(3,10):  # loops through different possible sequence lengths
            frequencies[student] = Counter(sequence[i:i+seq_length] for i in range(len(sequence)-seq_length-1))  # counts string matches for every string of the current length
    return frequencies

In [41]:
df2 = df[df.Activity == 'a3']
blocks = get_blocks(df2,get_students())
frequencies = get_frequencies(blocks)
all_frequencies = sum(frequencies.values(), Counter())

### Steps

Mine sequences
1. Find all sequences of length 3 to N per student per activity
* Calculate count per sequences
* Prune collection of sequences for those significant likely to be non-random

Analyze sequences
1. Intepret top 5 sequences
* Measure uniformity of sequences per condition
* Measure uniformity of sequences per cluster
* Measure correlation between use of sequence and pre to post gains


